# Requirements

In [1]:
import math
import sys

# Problem

We want to complete a knight's tour on a square board.  A knight's tour is a sequence of moves (like the knight in chess) on the board such that each position on the board is visited exactly once.

The $1 \times 1$ board is trivial, there is a single trivial tool.  It is immediately clear that there are no tours for $2 \times 2$ and $3 \times 3$ boards.  Maybe less trivial to see, there is also no tour for a $4 \times 4$ board.  An example of a knight's tour on a $8 \times 8$ board is given below:

$$
    \begin{array}{cccccccc}
        1 &  38 &  55 &  34 &   3 &  36 &  19 &  22 \\
       54 &  47 &   2 &  37 &  20 &  23 &   4 &  17 \\
       39 &  56 &  33 &  46 &  35 &  18 &  21 &  10 \\
       48 &  53 &  40 &  57 &  24 &  11 &  16 &   5 \\
       59 &  32 &  45 &  52 &  41 &  26 &   9 &  12 \\
       44 &  49 &  58 &  25 &  62 &  15 &   6 &  27 \\
       31 &  60 &  51 &  42 &  29 &   8 &  13 &  64 \\
       50 &  43 &  30 &  61 &  14 &  63 &  28 &   7 \\
    \end{array}
$$

# Implementation

We will use backtracking to solve this problem.  In order to keep track of the state, we define a class.

In [2]:
class Options:
    '''class to keep track of options to be checked in a backtracking
    algorithm.
    '''
    
    def __init__(self, options):
        '''create a new object that represents the options at a choice point
        
        Parameters
        ----------
        options: iterable[value_type]
            iterable that represents the potential options
        '''
        self._options = list(options)
        self._current_idx = 0
    
    @property
    def nr_options(self):
        '''number of options
        
        Returns
        -------
        int
            number of options that are available in total at this point
        '''
        return len(self._options)
    
    def has_option(self):
        '''check whether a next option is still available
        
        Returns
        -------
        bool
            True if more options are available, False otherwise
        '''
        return self._current_idx < self.nr_options
    
    @property
    def current_option(self):
        '''return the current option, and advance to the next option
        
        Returns
        -------
        value_type
            current option value, and advance to next
        Raises
        ------
        IndexError
            raised if no more options are available
        '''
        if self.has_option():
            option = self._options[self._current_idx]
            self._current_idx += 1
            return option
        else:
            raise IndexError(f'all options exhausted')
    
    def peek(self):
        '''get the current option without advancing to the next
        
        Returns
        -------
        value_type
            current option value
        Raises
        ------
        IndexError
            raised if either current_option hasn't been called yet, or no more options
            are available
        '''
        if 0 <= self._current_idx - 1 < self.nr_options:
            return self._options[self._current_idx - 1]
        elif self._current_idx - 1 < 0:
            raise IndexError('enumeration not initialized, first call current_option')
        else:
            raise IndexError('all options exhausted')

    def __repr__(self):
        '''compute string representation of the object
        
        Returns
        -------
        str
            string representation of the object's state
        '''
        return f'{self._current_idx} -> {self._options}'

The code below illustrates the use of such objects.

In [3]:
options = Options([3, 5, 7])
while options.has_option():
    print(options.current_option, end='')
    print(f' {options.peek()}')
options.peek()

3 3
5 5
7 7


7

In [4]:
options

3 -> [3, 5, 7]

For better performance, precompute the moves that are legal on each position on the board, not taking into account occupancy.

The positions of the board will be given by a single index ranging from $0$ to $n^2 - 1$, row-wise, i.e., for a $3 \times 3$ board:

$$
    \begin{array}{ccc}
        0 & 1 & 2 \\
        3 & 4 & 5 \\
        6 & 7 & 8 \\
    \end{array}
$$

In [5]:
def compute_all_moves(n):
    '''compute the possible moves on each position of the board
    
    Parameters
    ----------
    n: int
        dimension of the n-by-n board
        
    Returns
    -------
    list[list[int]]
        a list that contains lists of board positions; board positions are number row-wise from
        0 to n**2 - 1.
    '''
    deltas = [(-2, 1), (-1, 2), (1, 2), (2, 1), (2, -1), (1, -2), (-1, -2), (-2, -1)]
    all_moves = []
    for k in range(n**2):
        all_moves.append([])
        i, j = k//n, k % n
        for delta_i, delta_j in deltas:
            new_i, new_j = i + delta_i, j + delta_j
            if 0 <= new_i < n and 0 <= new_j < n:
                all_moves[-1].append(new_i*n + new_j)
    return all_moves

For instance, for a $4 \times 4$ board:

In [6]:
compute_all_moves(4)

[[6, 9],
 [7, 10, 8],
 [11, 9, 4],
 [10, 5],
 [2, 10, 13],
 [3, 11, 14, 12],
 [15, 13, 8, 0],
 [14, 9, 1],
 [1, 6, 14],
 [2, 7, 15, 0],
 [3, 12, 4, 1],
 [13, 5, 2],
 [5, 10],
 [6, 11, 4],
 [7, 8, 5],
 [9, 6]]

To check this, we can write a visualization function.

In [7]:
def visualize_option(all_moves, pos):
    '''visualize the possible moves at a given position
    
    Parameters
    ----------
    all_moves: list[list[int]]
        moves for each position
    pos: int
        position given as a number from 0 to n**2 - 1
    '''
    n = math.isqrt(len(all_moves))
    for i in range(n):
        for j in range(n):
            k = i*n + j
            if k == pos:
                print('x', end='')
            elif pos in all_moves[k]:
                print('o', end='')
            else:
                print('-', end='')
        print()

In [8]:
visualize_option(compute_all_moves(6), 21)

------
--o-o-
-o---o
---x--
-o---o
--o-o-


Given the symmetries of a $n \times n$ chess board, only a quarter of the possible first moves needs to be checked.  These are the potential starting points for the tour(s).

In [9]:
def compute_initial_moves(n):
    '''compute the relevant initial moves, taking into account
    the symmetry of the board
    
    Parameters
    ----------
    n: int
        size of the n-by-n board
    
    Returns
    -------
    list[int]
        positions to consider for the first move, i.e., the starting point of the
        tour
    '''
    return [i*n + j for i in range(n//2 + n % 2)
            for j in range(i, n//2 + n % 2)]

The initial positions for a $5 \times 5$ board are shown below.

In [10]:
compute_initial_moves(5)

[0, 1, 2, 6, 7, 12]

A tour will be a list of positions on the board.  In order to visualize that, we can use the function below.

In [11]:
def print_tour(tour):
    '''visualize a tour on the board
    
    Parameters
    ----------
    tour: list[int]
        the sequence of positions on the board
    '''
    n = math.isqrt(len(tour))
    board = [[0]*n for _ in range(n)]
    for i, pos in enumerate(tour):
        board[pos//n][pos % n] = i + 1
    for i in range(n):
        print(' '.join(map(lambda x: f'{x:4d}', board[i])))

In [12]:
def find_tour(n):
    '''find a knight's tour on an n-by-n board
    
    Parameters
    ----------
    n: int
        size of the n-by-n board
        
    Returns
    -------
    list[int]
        the sequence of positions on the board that form a valid knight's tour,
        or an empty list if no tour can be found
    '''
    all_moves = compute_all_moves(n)
    stack = [Options(compute_initial_moves(n))]
    tour = []
    while stack:
        is_updated = False
        while stack[-1].has_option():
            new_pos = stack[-1].current_option
            if new_pos not in tour:
                tour.append(new_pos)
                stack.append(Options(all_moves[new_pos]))
                is_updated = True
                break
        if len(tour) == n**2:
            return tour
        if not is_updated:
            stack.pop()
            if tour:
                tour.pop()
    return tour

Print a board with a knight's tour (if any) for board sizes $n \in \{1, \ldots, 8\}$:

In [13]:
for n in range(1, 9):
    print(f'------ {n} ----------')
    print_tour(find_tour(n))

------ 1 ----------
   1
------ 2 ----------
------ 3 ----------
------ 4 ----------
------ 5 ----------
   1   20   17   12    3
  16   11    2    7   18
  21   24   19    4   13
  10   15    6   23    8
  25   22    9   14    5
------ 6 ----------
   1   28   33   20    3   26
  34   19    2   27    8   13
  29   32   21   12   25    4
  18   35   30    7   14    9
  31   22   11   16    5   24
  36   17    6   23   10   15
------ 7 ----------
   1   28   37   24    3   26   17
  36   39    2   27   18   11    4
  29   42   23   38   25   16    9
  40   35   30   19   10    5   12
  43   22   41   32   15    8   47
  34   31   20   45   48   13    6
  21   44   33   14    7   46   49
------ 8 ----------
   1   38   55   34    3   36   19   22
  54   47    2   37   20   23    4   17
  39   56   33   46   35   18   21   10
  48   53   40   57   24   11   16    5
  59   32   45   52   41   26    9   12
  44   49   58   25   62   15    6   27
  31   60   51   42   29    8   13   64
  50 

We can easily write a function to verify whether a given tour is a valid knight's tour.

In [14]:
def is_valid_knights_tour(tour, is_verbose=False):
    '''check whether a given sequence of positions represents a valid
    knight's tour
    
    Parameters
    ----------
    tour: list[int]
        sequence of board positions
    is_verbose: bool
        if True, print the reason why the tour is not valid, default is False
    
    Returns
    -------
    bool
        True if the sequence is a knight's tour, False otherwise
    '''
    if len(tour) == 0:
        if is_verbose:
            print(f'length 0 tour')
        return False
    n = math.isqrt(len(tour))
    if len(tour) != n**2:
        if is_verbose:
            print(f'length of the tour does not fit on a squaure board {n}x{n}', file=sys.stderr)
        return False
    if len(set(tour)) != len(tour):
        if is_verbose:
            print(f'duplicate positions in tour')
        return False
    if min(tour) != 0 or max(tour) != n**2 - 1:
        if is_verbose:
            print(f'invalid positions in tour')
    deltas = [(-2, 1), (-1, 2), (1, 2), (2, 1), (2, -1), (1, -2), (-1, -2), (-2, -1)]
    for i, pos in enumerate(tour[1:]):
        if all(tour[i] != ((pos//n + delta_i)*n + pos % n + delta_j) for delta_i, delta_j in deltas):
            print(f'illegal move from {tour[i]} to {pos}')
            return False
    return True

Check that an invalid sequence is not valid.

In [15]:
is_valid_knights_tour(list(range(16)), True)

illegal move from 0 to 1


False

Check tours for $n \in \{1, \ldots, 8\}$.

In [16]:
for n in range(1, 9):
    print(f'------ {n} ----------')
    tour = find_tour(n)
    print(is_valid_knights_tour(tour))

------ 1 ----------
True
------ 2 ----------
False
------ 3 ----------
False
------ 4 ----------
False
------ 5 ----------
True
------ 6 ----------
True
------ 7 ----------
True
------ 8 ----------
True


# Performance

In [14]:
%timeit find_tour(5)

68.9 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [15]:
%timeit find_tour(6)

10.3 ms ± 61.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%timeit find_tour(7)

26.7 s ± 497 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit find_tour(8)

8.79 s ± 49.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The runtime increases rapidly.

# Conclusions

This implementation is reasonably fast.  In fact, it is much faster than the naive approach that woulld enumerate all permutations of the sequence $0, \ldots, n^2 - 1$, and checking for each whether it represents a valid knight's tour.